In [ ]:
from __future__ import annotations
from typing import TypedDict, Dict, Any
from pathlib import Path

# LangChain - Ollama
try:
    from langchain_ollama import ChatOllama
except ImportError:
    from langchain_community.chat_models import ChatOllama  # 舊版用這個

from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser

# LangGraph
from langgraph.graph import StateGraph, END


# ------------------------
# 載入本地 Prompt
# ------------------------
PROMPT_FILES = {
    "DevEnvAgent": Path("DevEnvAgent.txt"),
    "IssueAgent": Path("IssueAgent.txt"),
    "CodeTraceAgent": Path("CodeTraceAgent.txt"),
    "TestingAgent": Path("TestingAgent.txt"),
}

PROMPTS: Dict[str, str] = {k: v.read_text(encoding="utf-8") for k, v in PROMPT_FILES.items()}


# ------------------------
# LLM (Ollama)
# ------------------------
llm = ChatOllama(
    base_url="http://10.1.1.59:11434",   # 確保 ollama 服務在本機 11434
    model="gpt-oss:20b",   # 換成你本地可用的模型
    temperature=0.1,
)


# ------------------------
# Agent 工廠
# ------------------------
def make_agent(system_prompt: str):
    # 用 f-string 把文件內容塞進 system，不讓 LangChain 解析裡面的 {}
    template = """{system_prompt}

使用者的問題如下：
{user_input}
"""
    prompt = ChatPromptTemplate.from_template(template)
    chain = prompt.partial(system_prompt=system_prompt.strip()) | llm | StrOutputParser()
    return chain

AGENTS = {name: make_agent(prompt) for name, prompt in PROMPTS.items()}


# RouterAgent Prompt
ROUTER_PROMPT = """
你是一個問題分流 Agent。  
你的任務是根據使用者的輸入，判斷應該交給哪個專家 Agent 處理。  
請只輸出對應的 Agent 名稱，不要輸出多餘文字。  

可選的 Agent 類別有：
- DevEnvAgent → 開發環境、VSCode、Dev Container、iptables、dockerd、debug mode
- IssueAgent → GitHub Issue、Swarm 問題
- CodeTraceAgent → Code Trace、Container 啟動邏輯、containerd、gRPC
- TestingAgent → 測試、unit test、integration test、TESTFLAGS、bridge_test.go

輸出格式：
<AgentName>
"""

# 建立 Prompt 模板
router_prompt = ChatPromptTemplate.from_messages([
    ("system", ROUTER_PROMPT),
    ("human", "{question}")
])

router_chain = router_prompt | llm

def route_question(user_question: str) -> str:
    """呼叫 LLM 來判斷要分派的 Agent"""
    response = router_chain.invoke({"question": user_question})
    output = response.content
    print("=================Agent Select===========")
    print(output)
    print("=================Agent Select===========")
    return output
 

# ------------------------
# LangGraph 狀態定義
# ------------------------
class GraphState(TypedDict):
    user_input: str
    routed: str | None
    response: str


# ------------------------
# Graph 節點
# ------------------------
def router_node(state: GraphState) -> GraphState:
    agent_name = route_question(state["user_input"])
    return {**state, "routed": agent_name}

def agent_node(state: GraphState) -> GraphState:
    agent_name = state["routed"]
    chain = AGENTS[agent_name]
    output = chain.invoke({"user_input": state["user_input"]})
    return {**state, "response": output}


# ------------------------
# 建立 Graph
# ------------------------
graph = StateGraph(GraphState)
graph.add_node("router", router_node)
graph.add_node("agent", agent_node)

graph.set_entry_point("router")
graph.add_edge("router", "agent")
graph.add_edge("agent", END)

app = graph.compile()


# ------------------------
# 測試用函數
# ------------------------
def run_once(user_text: str) -> Dict[str, Any]:
    state = {"user_input": user_text, "routed": None, "response": ""}
    result = app.invoke(state)
    return {
        "routed_agent": result["routed"],
        "response": result["response"]
    }


# ------------------------
# CLI 互動
# ------------------------
if __name__ == "__main__":
    print("=== RouterAgent + 4 Agents (Ollama) ===")
    while True:
        q = input("\nYou > ").strip()
        if q.lower() in {"exit", "quit"}:
            break
        print("\033[34m "+q+" \033[0m")
        out = run_once(q)
        print(f"[Router → {out['routed_agent']}]")
        print("\033[34m"+out["response"]+"\033[0m")

=== RouterAgent + 4 Agents (Ollama) ===



You >  debug mode launch.json怎麼設定？json only


 debug mode launch.json怎麼設定？json only 
=================Agent Select===========
DevEnvAgent
=================Agent Select===========
[Router → DevEnvAgent]
```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Launch Package",
            "type": "go",
            "request": "launch",
            "mode": "auto",
            "program": "${fileDirname}"
        }
    ]
}
```



You >  debug mode launch.json怎麼設定？


 debug mode launch.json怎麼設定？ 
=================Agent Select===========
DevEnvAgent
=================Agent Select===========
[Router → DevEnvAgent]
**debug mode 的 `launch.json` 只需要一個簡單的 Go 調試配置即可。**

```json
{
    "version": "0.2.0",
    "configurations": [
        {
            "name": "Launch Package",
            "type": "go",
            "request": "launch",
            "mode": "auto",
            "program": "${fileDirname}"
        }
    ]
}
```

### 位置
- 把這段 JSON 放在專案根目錄下的 `.vscode/launch.json`（如果沒有 `.vscode` 資料夾，先建立一個）。

### 參數說明
| 參數 | 作用 |
|------|------|
| `name` | 調試配置的顯示名稱（可自行改成更具描述性的名稱）。 |
| `type` | 必須是 `go`，表示使用 Go 調試器。 |
| `request` | `launch` 表示啟動並進入調試。 |
| `mode` | `auto` 讓 VSCode 自動決定是使用 `debug` 還是 `test` 模式。 |
| `program` | `${fileDirname}` 代表目前打開檔案所在目錄的 Go package。若想調試整個專案，保持這個設定即可；若想指定其他目錄，只需改成相對或絕對路徑。 |

### 進一步設定（可選）
- 若需要傳遞環境變數或額外參數，可在配置中加入 `env` 或 `args` 欄位。
- 若想在容器內部執行，確保已經安裝 **Dev Containers** 插件、重新啟動 VSCode、並使用 `Reopen in Container` 指令。

只要把上述 `launch.json` 


You >  我想要做unit test, 該怎麼做


 我想要做unit test, 該怎麼做 
=================Agent Select===========
TestingAgent
=================Agent Select===========
[Router → TestingAgent]
要執行 Moby Project 的單元測試，請按下列步驟操作：

1. **進入專案根目錄**  
   確保你已經在專案的根目錄（即 `moby` 目錄）下。

2. **執行單元測試指令**  
   ```bash
   make test-unit
   ```
   這個指令會自動跑所有單元測試。

3. **查看測試結果**  
   測試完成後，結果會顯示在 **TEST RESULT** 面板中。  
   （在文件中附有測試結果截圖 `image.png`，可作為參考。）

> **備註**  
> - 若你只想跑單一測試，可以在 Debug 模式下執行 `daemon.go`，然後在 `bridge_test.go` 點擊綠色 ▶️ 按鈕執行。  
> - 以上步驟已在 `TESTING.md` 中說明，並附有相應截圖供你參考。
